# PRE-PROCESSING - TEST

The objective of this notebook consists of the elimination of variables previously selected by means of analysis of the data dictionary and data type by variable, together with the categorical variables' dummarization.

## SET UP

In [15]:
!pip install findspark

import findspark
findspark.init()

## LIBRARIES

In [16]:
# Cargar Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()

In [17]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## LOAD DATA

In [19]:
df = spark.read.option("header","true").option("inferschema","true").option("delimiter", ",").option("nullValue", "\\N").csv("../iteration_1/input/dt2020_base_evaluar_semicolon.csv")

In [20]:
df.count()

281666

In [21]:
df.limit(5).toPandas().head()

,id_registro,periodo,id_cli,fecha_nacimiento,edad,genero,estado_civil,nivel_academico,profesion,ocupacion,...,saldo_no_rot_mdo,cuota_libranza_sf,cant_oblig_tot_sf,cant_cast_ult_12m_sr,ind,rep_calif_cred,pol_centr_ext,convenio_lib,ingreso_nomina,ingreso_segurida_social
0,12154#201902,201902,12154,19430621,75.575633,M,CASADO,BACHILLER,None,Desempleado con Ingresos,...,262000,0,1,0,6.412382e+06,H,1,None,NaN,NaN
1,374#201902,201902,374,19810909,37.355236,F,SOLTERO,NO INFORMA,None,Empleado,...,0,0,1,0,1.323586e+05,C,0,None,NaN,NaN
2,7365#201902,201902,7365,19810311,37.853525,F,NO INFORMA,NO INFORMA,None,Empleado,...,0,0,1,0,1.394262e+06,D,0,None,NaN,NaN
3,16314#201902,201902,16314,19781030,40.216290,F,CASADO,UNIVERSITARIO,CONTADURIA,Empleado,...,8323000,0,0,44,3.483775e+06,H,8,None,NaN,NaN
4,19374#201902,201902,19374,19790903,39.373032,F,CASADO,ESPECIALIZACION,PSICOLOGIA,Profesional Independiente,...,0,0,12,0,5.554769e+07,A,0,None,NaN,NaN


## PRE-PROCESSING

### Imputations

In [22]:
data_imputation = df.fillna(0)

## Limpieza de variables

In [23]:
data_clean = data_imputation

In [24]:
# Estas son las que tienen X o \N, pero son flags, entonces hay que cambiar 
# X por 1
flags = ['tiene_consumo', 
         'tiene_crediagil', 
         'tiene_ctas_activas', 
         'tiene_ctas_embargadas', 
         'pension_fopep', 
         'tiene_cred_hipo_1', 
         'tiene_cred_hipo_2']

In [25]:
for var in flags:
    data_clean =  data_clean.withColumn(var, when(col(var) == "X", "1").otherwise(0))

In [26]:
# Estas son varibles con respuesta S, N
# ind_mora_vigente N, S, \N
# cartera_castigada N, S, \N
sn_vars = ['ind_mora_vigente', 'cartera_castigada']

In [27]:
for var in sn_vars:
    #print (flag)
    #data = df.withColumn(flag, when(col(flag) == 'X', 1))
    data_clean =  data_clean.withColumn(var, when(col(var) == "S", "1").otherwise(0))

In [28]:
data_clean =  data_clean.withColumn("tenencia_tc", when(col("tenencia_tc") == "SI", "1").otherwise(0))

In [29]:
var_drop = [
    'genero',
    'profesion',
    'ult_actual',
    'codigo_ciiu',
    'ciudad_residencia',
    'ciudad_laboral',
    'departamento_laboral',
    'convenio_lib']

In [30]:
# Drop 
data_clean = data_clean.drop(*var_drop)

In [31]:
# Estado civil
data_clean = data_clean.withColumn("estado_civil"
                                   ,when(col("estado_civil") == "VIUDO",4)
                                   .when(col("estado_civil") == "SOLTERO",1)
                                   .when(col("estado_civil") == "NO INFORMA",0)
                                   .when(col("estado_civil") == "OTRO",0)
                                   .when(col("estado_civil") == "CASADO",3)
                                   .when(col("estado_civil") == "DIVORCIADO",5)
                                   .when(col("estado_civil") == "UNION LIBRE",2)
                                   .otherwise(0)
                                  )

In [32]:
# Nivel academico
data_clean = data_clean.withColumn("nivel_academico"
                                   ,when(col("nivel_academico") == "SIN INFORMACION",4)
                                   .when(col("nivel_academico") == "BACHILLER",1)
                                   .when(col("nivel_academico") == "NO INFORMA",0)
                                   .when(col("nivel_academico") == "ESPECIALIZACION",0)
                                   .when(col("nivel_academico") == "NINGUNO",3)
                                   .when(col("nivel_academico") == "TECNOLOGO",5)
                                   .when(col("nivel_academico") == "UNIVERSITARIO",2)
                                   .when(col("nivel_academico") == "PRIMARIO",2)
                                   .otherwise(0)
                                  )


In [33]:
#Ocupación
data_clean = data_clean.withColumn("ocupacion"
                                   ,when(col('ocupacion')=='Empleado','Empleado')
                                   .when(col('ocupacion')=='Independiente','Independiente')
                                   .when(col('ocupacion')=='Estudiante','Estudiante')
                                   .when(col('ocupacion')=='Profesional Independiente','Independiente')
                                   .when(col('ocupacion')=='Jubilado','Pensionado')
                                   .when(col('ocupacion')=='Comerciante','Independiente')
                                   .when(col('ocupacion')=='Socio Empleado - Socio','Empleado')
                                   .when(col('ocupacion')=='Ama de Casa','Independiente')
                                   .when(col('ocupacion')=='Otro','Indefinida')
                                   .when(col('ocupacion')=='Rentista de Capital','Independiente')
                                   .when(col('ocupacion')=='Pensionado','Pensionado')
                                   .when(col('ocupacion')=='Desempleado sin Ingresos','Desempleado')
                                   .when(col('ocupacion')=='Sin Ocupacion Asignada','Indefinida')
                                   .when(col('ocupacion')=='Desempleado con Ingresos','Independiente')
                                   .when(col('ocupacion')=='Ganadero','Independiente')
                                   .when(col('ocupacion')=='Agricultor','Independiente')
                                   .otherwise("Indefinida")
                                  )

In [34]:
# Tipo de vivienda
data_clean = data_clean.withColumn("tipo_vivienda"
                                   ,when(col("tipo_vivienda") == "ALQUILADA",1)
                                   .when(col("tipo_vivienda") == "NO INFORMA",0)
                                   .when(col("tipo_vivienda") == "PROPIA",3)
                                   .when(col("tipo_vivienda") == "FAMILIAR",2)
                                   .otherwise(0)
                                  )

In [35]:
# Rechazo Crédito
data_clean = data_clean.withColumn('rechazo_credito',
                                   when(col('rechazo_credito')=='RECHAZO BANCO',1)
                                   .when(col('rechazo_credito')=='RECHAZO FACTORING',1)
                                   .when(col('rechazo_credito')=='RECHAZO SUFI',1)
                                   .when(col('rechazo_credito')=='RECHAZO BIZAGI',1)
                                   .otherwise(0)
                                  )

In [36]:
#  Categoria de ingreso
data_clean = data_clean.withColumn('cat_ingreso'
                                   ,when(col('cat_ingreso') == "NOM",1)
                                   .when(col('cat_ingreso') == "PEN",2)
                                   .when(col('cat_ingreso') == "NOM PEN",3)
                                   .otherwise(0)
                                  )

In [37]:
# rep_calif_cred
data_clean = data_clean.withColumn('rep_calif_cred'
                                   ,when(col('rep_calif_cred') == "A",1)
                                   .when(col('rep_calif_cred') == "B",2)
                                   .when(col('rep_calif_cred') == "C",3)
                                   .when(col('rep_calif_cred') == "D",4)
                                   .when(col('rep_calif_cred') == "E",5)
                                   .when(col('rep_calif_cred') == "F",6)
                                   .when(col('rep_calif_cred') == "G",7)
                                   .when(col('rep_calif_cred') == "H",8)
                                   .otherwise(0)
                                  )

In [38]:
# Departamento
data_clean = data_clean.withColumn('departamento_residencia'
                                   ,when(col('departamento_residencia')=='ANTIOQUIA                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='VALLE                                             ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='RISARALDA                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='SANTANDER                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='BOGOTA D.C.                                       ', 'ANDINA')
                                   .when(col('departamento_residencia')=='ESTADO EXTERIOR                                   ' , 'EXTERIOR')
                                   .when(col('departamento_residencia')=='QUINDIO                                           ', 'ANDINA')
                                   .when(col('departamento_residencia')=='BOLIVAR                                           ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CORDOBA                                           ', 'CARIBE')
                                   .when(col('departamento_residencia')=='NORTE DE SANTANDER                                ', 'ANDINA')
                                   .when(col('departamento_residencia')=='ATLANTICO                                         ', 'CARIBE')
                                   .when(col('departamento_residencia')=='MAGDALENA                                         ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='CUNDINAMARCA                                      ', 'ANDINA')
                                   .when(col('departamento_residencia')=='TOLIMA                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='SUCRE                                             ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CAUCA                                             ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='META                                              ' , 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='CESAR                                             ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='NARINO                                            ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='NI', 'NI')
                                   .when(col('departamento_residencia')=='BOYACA                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='CAQUETA                                           ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='CALDAS                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='HUILA                                             ', 'ANDINA')
                                   .when(col('departamento_residencia')=='CASANARE                                          ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='SAN ANDRES Y PROVID                               ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='NI                                   ', 'NI')
                                   .when(col('departamento_residencia')=='LA GUAJIRA                                        ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CHOCO                                             ', 'PACIFICO')
                                   .when(col('departamento_residencia')=='ARAUCA                                            ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='PUTUMAYO                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='GUANIA                                            ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='GUAVIARE                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='VAUPES                                            ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='AMAZONAS                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='VICHADA                                           ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='MADRID                                            ', 'EXTERIOR')
                                   .when(col('departamento_residencia')=='ESTADO DE LA FLORIDA                              ', 'EXTERIOR')
                                  )


In [39]:
data_dummy = data_clean

In [40]:
# Ocupación
data_dummy = (data_dummy
              .withColumn("Empleado",when(col("ocupacion")=="Empleado",1).otherwise(0))
              .withColumn("Independiente",when(col("ocupacion")=="Independiente",1).otherwise(0))
              .withColumn("Estudiante",when(col("ocupacion")=="Estudiante",1).otherwise(0))
              .withColumn("Pensionado",when(col("ocupacion")=="Pensionado",1).otherwise(0))
              .withColumn("Indefinida",when(col("ocupacion")=="Indefinida",1).otherwise(0))
              .withColumn("Desempleado",when(col("ocupacion")=="Desempleado",1).otherwise(0))
              .drop("ocupacion")
             )

In [41]:
# Departamento
data_dummy = (data_dummy
              .withColumn('ANDINA',when(col('departamento_residencia')=="ANDINA",1).otherwise(0))
              .withColumn('PACIFICO',when(col('departamento_residencia')=="PACIFICO",1).otherwise(0))
              .withColumn('EXTERIOR',when(col('departamento_residencia')=="EXTERIOR",1).otherwise(0))
              .withColumn('CARIBE',when(col('departamento_residencia')=="CARIBE",1).otherwise(0))
              .withColumn('ORINOQUIA',when(col('departamento_residencia')=="ORINOQUIA",1).otherwise(0))
              .withColumn('NI',when(col('departamento_residencia')=="NI",1).otherwise(0))
              .withColumn('AMAZONIA',when(col('departamento_residencia')=="AMAZONIA",1).otherwise(0))
              .drop("departamento_residencia")
             )

In [42]:
data_preprocessed = data_dummy

In [43]:
data_preprocessed.count()

281666

In [45]:
# Save as parquet file
data_preprocessed.write.parquet("output/preprocessing/preprocessing_data_test")